In [76]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# preprocessing
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler

# model selection
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score

from sklearn.metrics import (r2_score, mean_squared_error, accuracy_score, precision_score, recall_score, make_scorer,
                             f1_score, roc_auc_score, roc_curve, precision_recall_curve)

# models
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from datetime import date

from pandas.plotting import scatter_matrix


In [77]:
data = pd.read_csv('data/train.csv',index_col= 'SalesID')
data.info()
#Columnas relevantes


C:\Users\NGALAZ\AppData\Local\Temp\ipykernel_10380\3501506613.py:1: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/train.csv',index_col= 'SalesID')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 401125 entries, 1139246 to 6333342
Data columns (total 52 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalePrice                 401125 non-null  int64  
 1   MachineID                 401125 non-null  int64  
 2   ModelID                   401125 non-null  int64  
 3   datasource                401125 non-null  int64  
 4   auctioneerID              380989 non-null  float64
 5   YearMade                  401125 non-null  int64  
 6   MachineHoursCurrentMeter  142765 non-null  float64
 7   UsageBand                 69639 non-null   object 
 8   saledate                  401125 non-null  object 
 9   fiModelDesc               401125 non-null  object 
 10  fiBaseModel               401125 non-null  object 
 11  fiSecondaryDesc           263934 non-null  object 
 12  fiModelSeries             56908 non-null   object 
 13  fiModelDescriptor         71919 non-n

In [78]:
#quitar columnas no releventes 
working_df = data.drop(["MachineID"
                        ,"datasource"
                        ,"auctioneerID"
                        ,"UsageBand"
                        ,"saledate"
                        ,"fiProductClassDesc"
                        ,"ProductGroupDesc"
                        ,"fiBaseModel"
                        ,"fiModelDesc"
                        ,"fiSecondaryDesc"
                        ,"fiModelSeries"
                        ,"fiModelDescriptor"
                        ,"ProductGroupDesc"
                       ],axis=1)
working_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401125 entries, 1139246 to 6333342
Data columns (total 40 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalePrice                 401125 non-null  int64  
 1   ModelID                   401125 non-null  int64  
 2   YearMade                  401125 non-null  int64  
 3   MachineHoursCurrentMeter  142765 non-null  float64
 4   ProductSize               190350 non-null  object 
 5   state                     401125 non-null  object 
 6   ProductGroup              401125 non-null  object 
 7   Drive_System              104361 non-null  object 
 8   Enclosure                 400800 non-null  object 
 9   Forks                     192077 non-null  object 
 10  Pad_Type                  79134 non-null   object 
 11  Ride_Control              148606 non-null  object 
 12  Stick                     79134 non-null   object 
 13  Transmission              183230 non-

In [79]:
categorias = working_df["ProductSize"].unique()
obj_map={}
#train["Sex"] = train["Sex"].map({"male":0, "female":1})
#working_df["ProductSize"]= working_df["ProductSize"].map({cat:})
for idx,cat in enumerate(categorias):
    obj_map[cat]=idx 
print(obj_map)

working_df["ProductSize"]= working_df["ProductSize"].map(obj_map)


{nan: 0, 'Medium': 1, 'Small': 2, 'Large / Medium': 3, 'Mini': 4, 'Large': 5, 'Compact': 6}


In [64]:
def mapea_categorias(df):
    """Mapear categorias a valores enteros si la columna es objeto"""
    for column in df.columns:
        obj_map={}
        if(df[column].dtype != np.int64 and df[column].dtype != np.float64):
            for idx,cat in enumerate(df[column].unique()):
                obj_map[cat]=idx 
            df[column]=df[column].map(obj_map)
    return df



In [81]:
working_df = mapea_categorias(working_df)
working_df.head(20)

,SalePrice,ModelID,YearMade,MachineHoursCurrentMeter,ProductSize,state,ProductGroup,Drive_System,Enclosure,Forks,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
SalesID,,,,,,,,,,,,,,,,,,,,,
1139246,66000,3157,2004,68.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1139248,57000,77,1996,4640.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1139249,10000,7009,2001,2838.0,0,2,1,0,1,0,...,0,0,0,0,0,0,0,0,1,1
1139251,38500,332,2001,3486.0,2,3,2,0,0,1,...,0,0,0,0,0,0,0,0,1,1
1139253,11000,17311,2007,722.0,0,2,1,0,2,0,...,0,0,0,0,0,0,0,0,1,1
1139255,26500,4605,2004,508.0,0,4,3,1,1,0,...,0,0,0,0,0,0,0,0,1,1
1139256,21000,1937,1993,11540.0,3,5,2,0,2,1,...,1,1,1,1,1,0,0,0,1,1
1139261,27000,3539,2001,4883.0,0,6,3,1,1,0,...,0,0,0,0,0,0,0,0,1,1
1139272,21500,36003,2008,302.0,4,3,2,0,2,1,...,1,1,1,1,1,0,0,0,1,1


In [94]:
#Reducir predictores
working_df = working_df[["SalePrice","YearMade","ModelID","MachineHoursCurrentMeter","ProductSize","state","Transmission","Turbocharged","Engine_Horsepower"]]

In [95]:
working_df.isna().sum()

SalePrice                   0
YearMade                    0
ModelID                     0
MachineHoursCurrentMeter    0
ProductSize                 0
state                       0
Transmission                0
Turbocharged                0
Engine_Horsepower           0
dtype: int64

In [86]:
MachineHoursCurrentMeter = pd.to_numeric(working_df.MachineHoursCurrentMeter, errors='coerce')
working_df['MachineHoursCurrentMeter'] = MachineHoursCurrentMeter.fillna(MachineHoursCurrentMeter.mean())

In [70]:
sns.pairplot(working_df, corner=True, diag_kind='kde');

Error in callback <function flush_figures at 0x000001EDAC1B4790> (for post_execute):


KeyboardInterrupt: 

In [96]:
y = working_df.SalePrice 
x = working_df.drop(['SalePrice'],axis=1)

In [97]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [98]:
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [99]:
#evalua regresion lineal
parameters = {}
model = LinearRegression()
gs = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=1)
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


(0.137208051092151, {})

In [100]:
#Evalua Ridge
parameters = {'alpha': np.logspace(-5, 5)}
model = Ridge()
gs = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=1)
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


(0.13720806297897256, {'alpha': 54.286754393238596})

In [102]:
#Evalua lasso
parameters = {'alpha': np.logspace(-5, 5)}
model = Lasso()
gs = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=1)
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


(0.13720806451922934, {'alpha': 0.7906043210907702})

In [103]:
parameters = {'n_neighbors': range(2,100), 'metric': ['manhattan', 'euclidean']}
model = KNeighborsRegressor()
gs = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=1)
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 196 candidates, totalling 980 fits


(0.6962277950115302, {'metric': 'manhattan', 'n_neighbors': 6})

In [104]:
# Modelo Final
final = KNeighborsRegressor(metric= 'manhattan', n_neighbors=6).fit(X_train, y_train)

In [105]:
final.score(X_test, y_test)

0.7083269838248474

In [106]:
data_test = pd.read_csv('data/test.csv',index_col= 'SalesID')


In [107]:
working_test = data_test.drop(["MachineID"
                        ,"datasource"
                        ,"auctioneerID"
                        ,"UsageBand"
                        ,"saledate"
                        ,"fiProductClassDesc"
                        ,"ProductGroupDesc"
                        ,"fiBaseModel"
                        ,"fiModelDesc"
                        ,"fiSecondaryDesc"
                        ,"fiModelSeries"
                        ,"fiModelDescriptor"
                        ,"ProductGroupDesc"
                       ],axis=1)


In [110]:
working_test = mapea_categorias(working_test)
working_test = working_test[["YearMade","ModelID","MachineHoursCurrentMeter","ProductSize","state","Transmission","Turbocharged","Engine_Horsepower"]]

In [112]:
working_test.isna().sum()

YearMade                       0
ModelID                        0
MachineHoursCurrentMeter    6834
ProductSize                    0
state                          0
Transmission                   0
Turbocharged                   0
Engine_Horsepower              0
dtype: int64

In [113]:
MachineHoursCurrentMeter = pd.to_numeric(working_test.MachineHoursCurrentMeter, errors='coerce')
working_test['MachineHoursCurrentMeter'] = MachineHoursCurrentMeter.fillna(MachineHoursCurrentMeter.mean())


C:\Users\NGALAZ\AppData\Local\Temp\ipykernel_10380\1976406158.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_test['MachineHoursCurrentMeter'] = MachineHoursCurrentMeter.fillna(MachineHoursCurrentMeter.mean())


In [114]:
working_test.isna().sum()

YearMade                    0
ModelID                     0
MachineHoursCurrentMeter    0
ProductSize                 0
state                       0
Transmission                0
Turbocharged                0
Engine_Horsepower           0
dtype: int64

In [115]:
#Predicciones
working_test["PredictedPrice"]=final.predict(working_test)

C:\Users\NGALAZ\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(
C:\Users\NGALAZ\AppData\Local\Temp\ipykernel_10380\822537489.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_test["PredictedPrice"]=final.predict(working_test)


In [117]:
working_test.head(200)

,YearMade,ModelID,MachineHoursCurrentMeter,ProductSize,state,Transmission,Turbocharged,Engine_Horsepower,PredictedPrice
SalesID,,,,,,,,,
1222837,1000,1376,0.0,0,0,0,0,0,25333.333333
1222839,2006,36526,4412.0,0,1,0,0,0,31750.000000
1222841,2000,4587,10127.0,0,1,0,0,0,31750.000000
1222843,1000,1954,4682.0,0,1,0,0,0,31750.000000
1222845,2002,4701,8150.0,1,2,0,0,0,31750.000000
...,...,...,...,...,...,...,...,...,...
1223597,1000,3823,17782.0,2,5,0,0,0,31750.000000
1223626,2005,9580,2133.0,1,10,0,0,0,31750.000000
1223627,2006,9580,1847.0,1,10,0,0,0,31750.000000


In [119]:
working_test.to_csv('data/predictedPrice.csv')


In [120]:
"""¿Cómo se eligió el modelo?
    - se tomó el que obtuvo mejor score en el gridsearch al evaluar los modelos de regresion (linear,lasso,ridge,knn)
    
¿Cómo se evaluó y comparó vs otros modelos?
    - se utilizo gridsearch para encontrar el mejor modelo con sus parametros que dan un mejor resultado 
      
¿Cuál fue el rendimiento con los datos de prueba?
    - El KNN regressor final dio un score de 0.7083269838248474 . Parametros Distancia : Manhattan K: 6
    
¿Qué problemas presentó el conjunto de datos?
    - Muchas variables categoricas
    - Al no dominar el negocio es complicado elegir las variables más relevantes. 
    - Muchos datos y variables que hace que el gridsearch consuma mucho tiempo

"""


'¿Cómo se eligió el modelo?\n    - se tomó el que obtuvo mejor score en el gridsearch al evaluar los modelos de regresion (linear,lasso,ridge,knn)\n    \n¿Cómo se evaluó y comparó vs otros modelos?\n    - se utilizo gridsearch para encontrar el mejor modelo con sus parametros que dan un mejor resultado \n      \n¿Cuál fue el rendimiento con los datos de prueba?\n    - El KNN regressor final dio un score de 0.7083269838248474 . Parametros Distancia : Manhattan K: 6\n    \n¿Qué problemas presentó el conjunto de datos?\n    - Muchas variables categoricas\n    - Al no dominar el negocio es complicado elegir las variables más relevantes. \n    - Muchos datos y variables que hace que el gridsearch consuma mucho tiempo\n\n'